In [2]:
import pygame
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 400, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Flappy Bird')

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Clock for controlling the frame rate
clock = pygame.time.Clock()

# Bird settings
bird = pygame.Rect(50, 300, 30, 30)
bird_velocity = 0
gravity = 0.5

# Pipe settings
pipe_width = 50
pipe_gap = 150
pipe_velocity = -3
pipes = []

def create_pipe():
    height = random.randint(100, 400)
    top_pipe = pygame.Rect(WIDTH, 0, pipe_width, height)
    bottom_pipe = pygame.Rect(WIDTH, height + pipe_gap, pipe_width, HEIGHT - height - pipe_gap)
    return top_pipe, bottom_pipe

pipes.append(create_pipe())

# Create a neural network model
model = Sequential([
    Dense(10, input_dim=4, activation='relu'),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='mse')

def get_state(bird, pipes):
    bird_y = bird.y
    pipe_x = pipes[0][0].x
    pipe_gap_y = pipes[0][0].height
    return np.array([bird_y, bird_velocity, pipe_x, pipe_gap_y])

def get_reward(bird, pipes):
    if bird.colliderect(pipes[0][0]) or bird.colliderect(pipes[0][1]):
        return -1  # Collision penalty
    return 0.1  # Reward for staying alive

training_data = []

def train_model(model, epochs=10):
    states, rewards = zip(*training_data)
    states = np.array(states)
    rewards = np.array(rewards)
    model.fit(states, rewards, epochs=epochs)

def reset_game():
    global bird, bird_velocity, pipes
    bird = pygame.Rect(50, 300, 30, 30)
    bird_velocity = 0
    pipes = [create_pipe()]

def autonomous_play():
    state = get_state(bird, pipes)
    action = model.predict(state.reshape(1, -1), verbose=0)
    if action > 0.5:
        bird_velocity = -8

running = True
autonomous_mode = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if not autonomous_mode and event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            bird_velocity = -8

    if autonomous_mode:
        autonomous_play()

    # Bird movement
    bird_velocity += gravity
    bird.y += bird_velocity

    # Pipe movement
    for pipe in pipes:
        pipe[0].x += pipe_velocity
        pipe[1].x += pipe_velocity

    # Remove pipes off the screen
    if pipes[0][0].x < -pipe_width:
        pipes.pop(0)
        pipes.append(create_pipe())

    # Collision detection and reward calculation
    reward = get_reward(bird, pipes)
    state = get_state(bird, pipes)
    training_data.append((state, reward))

    if reward == -1:
        # Train the model with collected data
        train_model(model)
        # Reset the game
        reset_game()
        autonomous_mode = True  # Switch to autonomous mode

    # Clear the screen
    screen.fill(WHITE)

    # Draw the bird
    pygame.draw.rect(screen, BLACK, bird)

    # Draw the pipes
    for pipe in pipes:
        pygame.draw.rect(screen, BLACK, pipe[0])
        pygame.draw.rect(screen, BLACK, pipe[1])

    pygame.display.flip()
    clock.tick(30)

pygame.quit()


Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0103  
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0101
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0102
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0104
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0103
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0109
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0105
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0101
Epoch 9/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0120
Epoch 10/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0117
